In [12]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, losses
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, Cropping2D

import pandas as pd
from keras.utils import np_utils

from PIL import Image
import numpy as np

In [13]:
data = pd.read_csv('train_concepts.txt', delimiter=":", header=None)

In [14]:
data.head()

,0,1
0,C0738276,btt-8-083Fig3;abcd-28-01-0087-g03
1,C1287716,ipej030157-10
2,C0576088,aps-43-590-g001;aps-43-590-g002;aps-43-590-g00...
3,C2332719,CRIRH2016-2019250.001
4,C0796089,kjlm-31-49-g001;MGG3-4-599-g001;JPN-6-19-g004;...


In [87]:
res = []
for index, row in data[:50].iterrows():
#     print index
    for i in row[1].split(';'):
#         print str(i)+';'+str(row[0])+';'+str(index)
        res.append(str(i)+';'+str(row[0])+';'+str(index))        

In [88]:
final_results = pd.DataFrame(
    {
        "0":res,
    })
print final_results

                                                 0
0                         btt-8-083Fig3;C0738276;0
1                   abcd-28-01-0087-g03;C0738276;0
2                         ipej030157-10;C1287716;1
3                       aps-43-590-g001;C0576088;2
4                       aps-43-590-g002;C0576088;2
5                       aps-43-590-g005;C0576088;2
6                       aps-43-590-g006;C0576088;2
7                        JCIS-2-66-g002;C0576088;2
8                        cios-4-77-g001;C0576088;2
9     CRIM.ENDOCRINOLOGY2013-190962.002;C0576088;2
10        CRIM.RADIOLOGY2013-235209.001;C0576088;2
11                         prm-20-229-1;C0576088;2
12             10195_2011_165_Fig1_HTML;C0576088;2
13             10195_2011_165_Fig2_HTML;C0576088;2
14                      JCAS-6-171-g001;C0576088;2
15                     1476-0711-4-18-2;C0576088;2
16                   eplasty14ic11_fig1;C0576088;2
17                    kjim-14-2-94-16f4;C0576088;2
18                 JMedLife-04-

In [89]:
final_results.to_csv('Final_prediction1000.csv',  index = False, header=False)

In [90]:
now = pd.read_csv('/Users/sharath/Desktop/GP/sharath/Final_prediction1000.csv', delimiter=";", header=None)
k = now[[2]].values
res = np_utils.to_categorical(k)
# print res[:5]
leng =  res.shape[1]
print len(now[0]), len(res)

dataset_size  = len(now[0])

# print now.head()

labels = pd.DataFrame(res)
# print labels.head()


result = pd.concat([now, labels], axis=1, join_axes=[now.index])
# print result.head()
# result.to_csv("preprocessed.txt", header=False)


1673 1673


In [91]:
imgdim = 80

In [92]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape = (imgdim, imgdim,3),border_mode = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(leng, activation = 'softmax')) 

# model = Sequential()
# model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(64,64,3)))
# model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
# model.add(Flatten())
# # model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 80, 80, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               25600500  
_________________________________________________________________
dense_14 (Dense)             (None, 50)                25050     
Total params: 25,626,446
Trainable params: 25,626,446
Non-trainable params: 0
_________________________________________________________________


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(80, 80, 3...)`
  


In [93]:
# diffrent working method 

datagen = ImageDataGenerator(rescale=1. / 255)
x = []
y = []
for index, row in now.iterrows():
#     print index
    name = row.values[0] 
#     y.append(res[index])

    img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ name +".jpg")
    img = img.resize((imgdim,imgdim)).convert('RGB')
    temp = np.array(img)
#     print temp.shape
    x.append(temp)
#     yield (x, y)
# print x[0]
X = np.array(x)
print X.shape
print res.shape

datagen.fit(X)
# model.fit_generator(datagen.flow(X,res, shuffle=True),steps_per_epoch=dataset_size, epochs=1)


(1673, 80, 80, 3)
(1673, 50)


In [ ]:
for e in range(10):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(X,res, shuffle=True ,batch_size=100):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

('Epoch', 0)
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 5.4516 - acc: 0.4300
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 7.7367 - acc: 0.5200
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 7.5755 - acc: 0.5300
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 7.7367 - acc: 0.5200
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 8.6723 - acc: 0.4600
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 9.0261 - acc: 0.4400
Epoch 1/1
100/100 [==============================] - 2s 24ms/step - loss: 8.0590 - acc: 0.5000
Epoch 1/1
100/100 [==============================] - 3s 26ms/step - loss: 6.9308 - acc: 0.5700
Epoch 1/1


In [80]:

T = []
name = 'IJEM-16-371-g004'
img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ name +".jpg")
img = img.resize((imgdim,imgdim)).convert('RGB')
T.append(np.array(img))
# print test.shape
X = np.array(T)
X.shape

(1, 80, 80, 3)

In [81]:
model.predict(X)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [28]:
res

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [1]:
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(x_train)

# # # fits the model on batches with real-time data augmentation:
# # model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
# #                     steps_per_epoch=len(x_train) / 32, epochs=epochs)

# # here's a more "manual" example
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break


In [2]:
 # def generate_arrays_from_file():
    
#     while 1:
#         f = open(path)
#         for line in f:
#             # create numpy arrays of input data
#             # and labels, from each line in the file
#             x, y = process_line(line)
#             img = load_images(x)
#             yield (img, y)
#         f.close()

        
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='10',epochs=10)


# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')

In [3]:

# T = []
# name = 'aps-43-590-g001'
# img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ name +".jpg")
# img = img.resize((64,64)).convert('RGB')
# T.append(np.array(img))
# # print test.shape
# X = np.array(T)
# X.shape

In [4]:


# def generate_arrays_from_file():
#     for index, row in now.iterrows():
#         print index
#         x = row.values[0]
#         y = res[index]
#         img = Image.open("/Users/sharath/Desktop/GP/CaptionTraining2018/"+ x +".jpg")
#         T.append(np.array(img))
#         X = np.array(T)
#     yield (X, y)
    
# model.fit_generator(generate_arrays_from_file(),steps_per_epoch='1',epochs=1)






# log_dir = './tf-log/'
# tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# cbks = [tb_cb]

# target_dir = './models/'
# if not os.path.exists(target_dir):
#   os.mkdir(target_dir)
# model.save('./models/model.h5')
# model.save_weights('./models/weights.h5')

In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# #from sklearn.tree import DecisionTreeClassifier

# #run classification
# clf=RandomForestClassifier()
# clf.fit(X_train,y_train)

# #now, make predictions from the classifier
# y_predicts=clf.predict(X_val)
# acc = accuracy_score(y_val, y_predicts)
# print "Val acc: ", round(acc,3)